## Explainable Deep Learning

**Name: Ilseop(Shawn) Lee**

### Instructions
In this assignment, you will work with pretrained deep learning models to investigate model explainability in computer vision. Your objective is to apply GradCAM and at least two of its variants to a meaningful image classification problem of your choice and analyze *how* and *why* the model makes its decisions.

You are encouraged to select an image classification task that holds personal or societal significance. Potential areas include, but are not limited to: wildlife conservation, road safety, public health, environmental sustainability, or social impact. You may use an existing public dataset or a curated subset, and pretrained models such as ResNet-50 or Vision Transformers (ViT).


**Tasks**
Choose an image classification problem relevant to you (e.g., wildlife detection, object recognition in autonomous driving, or recycling classification).<br>
Use a pretrained computer vision model (e.g., ResNet-50, ViT) for your classification task. Transfer learning is optional.<br>
Apply Explainability Techniques:<br>
- Implement GradCAM and at least two GradCAM variants
- Apply these techniques to at least 5 images from your dataset
- Generate and present visualizations showing what regions of the image the model is focusing on for its predictions.
- Compare and contrast the attention maps generated by GradCAM and its variants.

Reflection:
- Discuss the visual cues the model attends to
- Comment on any surprising or misleading behavior
- Reflect on why model explainability is important in your selected application domain
----

- **Rooftop Solar Energy System Detection in Cape Town**
    - Task : Detect and classify Solar Energy Systems (Solar Panel, Water Heater, Pool Heater) from aerial imagery
    - Purpose : Assess spatial adoption patterns of solar energy system and provide evicende to suppoert energy policy and planning in Cape Town

*Note: This was my personal project('Energy Transition During Energy Crisis: Cape Town's Experience'). Therefore, I used the dataset and model that I had processed for that project*

-----

**Model and Data Preparation**
- Pre-trained Segmentation Mdoel
    - ResNext-50 encoder with Feature Pyramid Network Decoder
    - Loaded from a best performing checkpoint in ckpt format

- Dataset
    - Aerial Imagery from Cape Town (Cropped to 320*320)

- Classification Task
    - Class 0: Background, Class 1: Solar_Panel, Class 2: Water_heater, Class 3: Pool_heater

- Visualization
    - For Clarity, Solar Panels = 'Green', Water heater = 'Red', and Pool heater = 'Blue'.

In [10]:
import os, cv2, torch, numpy as np
from torch.utils.data import DataLoader, Dataset as BaseDataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch.nn as nn
import pytorch_lightning as pl
import segmentation_models_pytorch as smp
from torch.optim import lr_scheduler

# Class mapping
CLASSES = ["background", "Solar_Panel", "Water_heater", "Pool_heater"]
COLORS = {0:(0,0,0), 1:(0,255,0), 2:(255,0,0), 3:(0,0,255)}

# Paths
CKPT_PATH = "/Users/ilseoplee/XAI_AIPI590.01_2025Fall/Week6_artifacts/pv-model-epoch=38-valid_avg_PV_iou=0.9572.ckpt"
IMG_DIR   = "/Users/ilseoplee/XAI_AIPI590.01_2025Fall/Week6_artifacts/Test_Images_Prd"
OUT_BASE  = "/Users/ilseoplee/XAI_AIPI590.01_2025Fall/Week6_artifacts"

# ---------------- Model ----------------
class PVModel(pl.LightningModule):
    def __init__(self, arch, encoder_name, in_channels, out_classes, **kwargs):
        super().__init__()
        self.model = smp.create_model(
            arch, encoder_name=encoder_name, in_channels=in_channels, classes=out_classes, **kwargs
        )
        params = smp.encoders.get_preprocessing_params(encoder_name)
        self.register_buffer("std", torch.tensor(params["std"]).view(1,3,1,1))
        self.register_buffer("mean", torch.tensor(params["mean"]).view(1,3,1,1))
    def forward(self, image):
        image = (image - self.mean) / self.std
        return self.model(image)

# ---------------- Dataset ----------------
def get_validation_augmentation():
    return A.Compose([
        A.PadIfNeeded(min_height=320, min_width=320, border_mode=0),
        A.CenterCrop(height=320, width=320),
        ToTensorV2(),
    ])

class InferenceDataset(BaseDataset):
    def __init__(self, image_dir, augmentation=None):
        self.image_paths = [os.path.join(image_dir,f) for f in os.listdir(image_dir)]
        self.augmentation = augmentation
    def __len__(self): return len(self.image_paths)
    def __getitem__(self,i):
        img_path = self.image_paths[i]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if image.shape[:2] != (320,320):
            image = cv2.resize(image,(320,320))
        if self.augmentation:
            sample = self.augmentation(image=image)
            image = sample["image"]
        else:
            image = torch.from_numpy(image.transpose(2,0,1)).float()
        return image, str(img_path)

# ---------------- Model loading ----------------
device = "cuda" if torch.cuda.is_available() else "cpu"
model = PVModel.load_from_checkpoint(
    CKPT_PATH,
    arch="FPN",
    encoder_name="resnext50_32x4d",
    in_channels=3,
    out_classes=len(CLASSES),
    strict=False
).to(device)
model.eval()

dataset = InferenceDataset(IMG_DIR, augmentation=get_validation_augmentation())
loader  = DataLoader(dataset, batch_size=1, shuffle=False)

target_layer = model.model.encoder.layer4[-1].conv3
print("Model and dataset are ready.")


Model and dataset are ready.


**Grad-CAM**

In [11]:
# ---------------- Grad-CAM ----------------
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients, self.activations = None, None
        self.fwd_hook = target_layer.register_forward_hook(self.save_activation)
        self.bwd_hook = target_layer.register_full_backward_hook(self.save_gradient)

    def save_activation(self, module, input, output):
        self.activations = output.detach()

    def save_gradient(self, module, grad_input, grad_output):
        self.gradients = grad_output[0].detach()

    def __call__(self, input_tensor, target_class):
        logits = self.model(input_tensor)
        target = logits[:, target_class, :, :].mean()
        self.model.zero_grad()
        target.backward(retain_graph=True)

        # Compute class activation map
        weights = self.gradients.mean(dim=(2, 3), keepdim=True)
        cam_map = (weights * self.activations).sum(dim=1, keepdim=True)
        cam_map = torch.relu(cam_map).squeeze().cpu().numpy()
        cam_map = (cam_map - cam_map.min()) / (cam_map.max() + 1e-8)
        return cam_map, logits

def overlay_heatmap(img, cam_map, alpha=0.5):
    cam_map_resized = cv2.resize(cam_map, (img.shape[1], img.shape[0]))
    heatmap = cv2.applyColorMap(np.uint8(255 * cam_map_resized), cv2.COLORMAP_JET)
    return cv2.addWeighted(img, 1 - alpha, heatmap, alpha, 0)

# Output directory
outdir_gc = os.path.join(OUT_BASE, "gradcam_results")
os.makedirs(outdir_gc, exist_ok=True)

gradcam = GradCAM(model, target_layer)

# Generate Grad-CAM visualizations
for imgs, img_paths in loader:
    imgs = imgs.float().to(device)
    img_path = img_paths if isinstance(img_paths, str) else img_paths[0]
    orig = cv2.imread(img_path)

    for cls_idx in [1, 2, 3]:
        cam_map, _ = gradcam(imgs, cls_idx)
        overlay = overlay_heatmap(orig, cam_map)
        base = os.path.splitext(os.path.basename(img_path))[0]
        cv2.imwrite(os.path.join(outdir_gc, f"{base}_gradcam_class{cls_idx}.png"), overlay)

print(f"Grad-CAM visualizations saved to: {outdir_gc}")


Grad-CAM visualizations saved to: /Users/ilseoplee/XAI_AIPI590.01_2025Fall/Week6_artifacts/gradcam_results


**Grad-CAM++**

In [12]:
# ---------------- Grad-CAM++ ----------------
class GradCAMPlusPlus(GradCAM):
    def __call__(self, input_tensor, target_class):
        logits = self.model(input_tensor)
        target = logits[:, target_class, :, :].mean()
        self.model.zero_grad()
        target.backward(retain_graph=True)

        grads, acts = self.gradients, self.activations
        grads2, grads3 = grads ** 2, grads ** 3

        numerator = grads2
        denominator = 2 * grads2 + (acts * grads3).sum(dim=(2, 3), keepdim=True) + 1e-8
        alphas = numerator / denominator
        weights = (alphas * torch.relu(grads)).sum(dim=(2, 3), keepdim=True)

        # Compute Grad-CAM++ map
        cam_map = (weights * acts).sum(dim=1, keepdim=True)
        cam_map = torch.relu(cam_map).squeeze().cpu().numpy()
        cam_map = (cam_map - cam_map.min()) / (cam_map.max() + 1e-8)
        return cam_map, logits

# Output directory
outdir_pp = os.path.join(OUT_BASE, "gradcam_results_plusplus")
os.makedirs(outdir_pp, exist_ok=True)

gradcam_pp = GradCAMPlusPlus(model, target_layer)

# Generate Grad-CAM++ visualizations
for imgs, img_paths in loader:
    imgs = imgs.float().to(device)
    img_path = img_paths if isinstance(img_paths, str) else img_paths[0]
    orig = cv2.imread(img_path)

    for cls_idx in [1, 2, 3]:
        cam_map, _ = gradcam_pp(imgs, cls_idx)
        overlay = overlay_heatmap(orig, cam_map)
        base = os.path.splitext(os.path.basename(img_path))[0]
        cv2.imwrite(os.path.join(outdir_pp, f"{base}_gradcampp_class{cls_idx}.png"), overlay)

print(f"Grad-CAM++ visualizations saved to: {outdir_pp}")


Grad-CAM++ visualizations saved to: /Users/ilseoplee/XAI_AIPI590.01_2025Fall/Week6_artifacts/gradcam_results_plusplus


**Guided_Grad_CAM**

In [13]:
# ---------------- Guided Grad-CAM ----------------
import copy

# 1) Define Guided Backpropagation ReLU
class GuidedBackpropReLU(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        return torch.relu(input)

    @staticmethod
    def backward(ctx, grad_output):
        (input,) = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        grad_input[grad_output < 0] = 0
        return grad_input

# 2) Replace all ReLUs in the model with GuidedBackpropReLU
def replace_relu_with_guided(module):
    for name, child in module.named_children():
        if isinstance(child, nn.ReLU):
            module._modules[name] = nn.ReLU(inplace=False)
            module._modules[name].forward = lambda x: GuidedBackpropReLU.apply(x)
        else:
            replace_relu_with_guided(child)

# 3) Create a copy of the model with guided ReLUs
guided_model = copy.deepcopy(model)
replace_relu_with_guided(guided_model)
guided_model.eval()

# 4) Use the same Grad-CAM instance
gradcam_guided = GradCAM(model, target_layer)

# 5) Output directory
outdir_guided = os.path.join(OUT_BASE, "gradcam_guided")
os.makedirs(outdir_guided, exist_ok=True)

# 6) Run Guided Grad-CAM
for imgs, img_paths in loader:
    imgs = imgs.float().to(device)
    img_path = img_paths if isinstance(img_paths, str) else img_paths[0]
    orig = cv2.imread(img_path)

    for cls_idx in [1, 2, 3]:
        # Compute CAM
        cam_map, _ = gradcam_guided(imgs, cls_idx)

        # Guided Backpropagation
        imgs.requires_grad = True
        logits = guided_model(imgs)
        target = logits[:, cls_idx, :, :].mean()
        guided_model.zero_grad()
        target.backward(retain_graph=True)

        guided_grad = imgs.grad.detach().cpu().numpy()[0].transpose(1, 2, 0)
        guided_grad = (guided_grad - guided_grad.min()) / (guided_grad.max() + 1e-8)

        # Guided Grad-CAM = Guided Backprop * CAM
        cam_resized = cv2.resize(cam_map, (guided_grad.shape[1], guided_grad.shape[0]))
        guided_cam = guided_grad * cam_resized[..., np.newaxis]
        guided_cam = (guided_cam - guided_cam.min()) / (guided_cam.max() + 1e-8)
        guided_cam = np.uint8(255 * guided_cam)

        # Save visualization
        base = os.path.splitext(os.path.basename(img_path))[0]
        cv2.imwrite(os.path.join(outdir_guided, f"{base}_guidedcam_class{cls_idx}.png"), guided_cam)

print(f"Guided Grad-CAM visualizations saved to: {outdir_guided}")


Guided Grad-CAM visualizations saved to: /Users/ilseoplee/XAI_AIPI590.01_2025Fall/Week6_artifacts/gradcam_guided


![sanple1](/Users/ilseoplee/XAI_AIPI590.01_2025Fall/Week6_artifacts/picture22.png)